# MAPA

### Conexión

Volvemos a generar una conexió y un stream.

In [0]:
# Extraemos los datos del flujo de datos.
vuelos = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
    .option("subscribe", "proyecto") \
    .load()

vuelos.printSchema()

vuelos_string = vuelos.selectExpr("CAST(value AS STRING)", "timestamp")

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

# Definimos la estructura.

schema_string = "flight STRING,lat DOUBLE,lon DOUBLE, status STRING, from STRING"


Datos recientes

In [0]:
# Obtenemos los datos mediante una consulta

df2 = vuelos_string \
         .select(from_csv(col("value"), schema_string) \
         .alias("flight"), "timestamp")


df3 = df2.select("flight.*", "timestamp")#.filter(col('status').contains('L'))

df3.createOrReplaceTempView("flights");

find_text = spark.sql("SELECT * FROM flights")

reciente_stream = find_text \
            .writeStream \
            .trigger(processingTime='5 seconds') \
            .outputMode("append") \
            .option("truncate", "false") \
            .format("memory") \
            .queryName("recientes") \
            .start()

    
reciente_stream.awaitTermination(1)

Out[3]: False

In [0]:
# Visualizamos los datos.
reciente = spark.sql("SELECT * FROM recientes")
reciente.show(10)

+-------+-------+---------+------+----+--------------------+
| flight|    lat|      lon|status|from|           timestamp|
+-------+-------+---------+------+----+--------------------+
|AMX004D|40.4734| -75.7408|     A| CDG|2022-05-19 15:04:...|
| ANA180|44.1834|-137.7237|     A| NRT|2022-05-19 15:04:...|
|CLX6684|52.4367| -71.0162|     A| LUX|2022-05-19 15:04:...|
|   AMX2|49.5583| -62.9752|     A| MAD|2022-05-19 15:04:...|
| AFR178|57.1537| -54.8061|     A| CDG|2022-05-19 15:04:...|
|IBE6403|39.7254| -30.7396|     A| MAD|2022-05-19 15:04:...|
| DLH498|61.0893| -24.0759|     A| FRA|2022-05-19 15:04:...|
| AMX049| 4.1426| -83.6855|     A| LIM|2022-05-19 15:04:...|
| DAL577|20.5803| -97.7804|     A| ATL|2022-05-19 15:04:...|
|UAL1063|37.1525| -85.7573|     A| EWR|2022-05-19 15:04:...|
+-------+-------+---------+------+----+--------------------+
only showing top 10 rows



Instalamos Folium para visualizar un mapa.

In [0]:
!pip install folium

     |████████████████████████████████| 95 kB 2.6 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
# Construcción del Mapa:

import folium
import time

# Inicializamos el MAPA
my_map = folium.Map(
    location = [19.436111, -99.071944],
    start_zoom = 7,
    max_zoom = 7
)

folium.TileLayer('cartodbdark_matter').add_to(my_map)

folium.CircleMarker(
        location = [19.436111, -99.071944],
        radius=10, color = 'red'
    ).add_to(my_map)

Out[6]: <folium.vector_layers.CircleMarker at 0x7fa34c8d4bb0>

Dado que no fue posible la generación de un mapa dinamíco vamos a gráficar cada punto a partir de un vuelo, sobre el mapa centrado en el aeropuerto interacional de la Ciudad de México.

In [0]:
import webbrowser

# Consulta:
status = spark.sql("SELECT * FROM recientes")
# Graficación:
for vuelo in status.rdd.collect():
     folium.CircleMarker(
                location = [vuelo['lat'], vuelo['lon']],
                radius=1
                          ).add_to(my_map)
# Visualización
my_map

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<head> 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 
 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>
 
 <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>
 <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-1.12.4.min.js"></script>
 <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css"/>
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_df930e4268750aa9ad5f09945aa30242 {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 </style>
 
</head>
<body> 
 
 <div class="folium-map" id="map_df930e4268750aa9ad5f09945aa30242" ></div>
 
</body>
<script> 
 
 var map_df930e4268750aa9ad5f09945aa30242 = L.map(
 "map_df930e4268750aa9ad5f09945aa30242",
 {
 center: [19.436111, -99.071944],
 crs: L.CRS.EPSG3857,
 zoom: 10,
 zoomControl: true,
 preferCanvas: false,
 startZoom: 7,
 }
 );

 

 
 
 var tile_layer_18bfba3a34730a0fdc5ce74e219db967 = L.tileLayer(
 "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
 {"attribution": "Data by \u0026copy; \u003ca href=\"http://openstreetmap.org\"\u003eOpenStreetMap\u003c/a\u003e, under \u003ca href=\"http://www.openstreetmap.org/copyright\"\u003eODbL\u003c/a\u003e.", "detectRetina": false, "maxNativeZoom": 7, "maxZoom": 7, "minZoom": 0, "noWrap": false, "opacity": 1, "subdomains": "abc", "tms": false}
 ).addTo(map_df930e4268750aa9ad5f09945aa30242);
 
 
 var tile_layer_c5b07d7b07fd5a2b195b4da8ad087341 = L.tileLayer(
 "https://cartodb-basemaps-{s}.global.ssl.fastly.net/dark_all/{z}/{x}/{y}.png",
 {"attribution": "\u0026copy; \u003ca href=\"http://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors \u0026copy; \u003ca href=\"http://cartodb.com/attributions\"\u003eCartoDB\u003c/a\u003e, CartoDB \u003ca href =\"http://cartodb.com/attributions\"\u003eattributions\u003c/a\u003e", "detectRetina": false, "maxNativeZoom": 18, "maxZoom": 18, "minZoom": 0, "noWrap": false, "opacity": 1, "subdomains": "abc", "tms": false}
 ).addTo(map_df930e4268750aa9ad5f09945aa30242);
 
 
 var circle_marker_feb1584976b3312d9e5f299031e3e62c = L.circleMarker(
 [19.436111, -99.071944],
 {"bubblingMouseEvents": true, "color": "red", "dashArray": null, "dashOffset": null, "fill": false, "fillColor": "red", "fillOpacity": 0.2, "fillRule": "evenodd", "lineCap": "round", "lineJoin": "round", "opacity": 1.0, "radius": 10, "stroke": true, "weight": 3}
 ).addTo(map_df930e4268750aa9ad5f09945aa30242);
 
 
 var circle_marker_76bca08950b814d6213ad72d18dbb11e = L.circleMarker(
 [40.4734, -75.7408],
 {"bubblingMouseEvents": true, "color": "#3388ff", "dashArray": null, "dashOffset": null, "fill": false, "fillColor": "#3388ff", "fillOpacity": 0.2, "fillRule": "evenodd", "lineCap": "round", "lineJoin": "round", "opacity": 1.0, "radius": 1, "stroke": true, "weight": 3}
 ).addTo(map_df930e